# 데이터 전처리

## 1) 먼저 “대회 요구사항”을 전처리/모델 설계에 맞게 번역하기
    제출/평가 핵심

        각 test RNA 서열(target)마다 3D 구조를 5개 제출해야 하고, 각 target의 (TM-score 기준) best-of-5를 평균낸 값이 최종 점수입니다.

        Part 2는 “잘 맞는 부분만”으로 점수를 벌기 어렵도록, 번호(잔기 인덱스)가 맞게 정렬/대응되는 잔기가 중요하다는 점이 강조되어 있습니다.

        **노트북 제출 방식(code competition)**이라서, 노트북이 test_sequences.csv를 읽고 submission.csv를 만들어야 합니다.

    데이터 파일/컬럼 구조(중요 전처리 포인트)

        train/validation/test_sequences.csv는 보통 target_id, sequence 외에 temporal_cutoff, description, stoichiometry, all_sequences, ligand_ids, ligand_SMILES 같은 메타 컬럼이 함께 있습니다.

        train_labels.csv(및 validation_labels.csv)의 ID는 target_id와 residue number를 _로 연결한 문자열입니다. 즉 ID = "{target_id}_{resid}" 형태.

        좌표는 **RNA의 C1′ 원자 좌표(x,y,z)**를 다루는 것으로 안내/베이스라인들이 작성되어 있습니다.

        sample_submission.csv는 train_labels와 “행 단위 포맷”은 동일하지만, 5개 구조를 제출해야 해서 x_1,y_1,z_1 ... x_5,y_5,z_5처럼 좌표 컬럼이 5세트 들어갑니다.

        입력 데이터 폴더에 MSA 파일들이 함께 제공되는 사례가 많고(EDA 로그에서 MSA 파일 수가 언급됨), 템플릿 접근용 **PDB_RNA/**도 제공됩니다.

## 2) 전처리에서 점수에 직접 영향 주는 “체크리스트”
### (A) ID 파싱 & residue 정렬은 무조건 검증

    TM-score 기반 구조 비교에서 **잔기 대응(순서/번호)**가 어긋나면 모델이 좋아도 점수가 크게 손해봅니다. 그래서 아래를 항상 검사하세요.

        ID를 rsplit('_', 1)로 나눠서 (target_id, resid)를 얻는다.

        각 target_id 그룹에서 resid 기준으로 정렬 후,

        resid가 1부터 L까지 연속인지

        resname(있다면)와 sequence[resid-1]가 일치하는지

        좌표 행 개수 == 서열 길이인지를 assert로 걸어두는 게 좋습니다.

### (B) “temporal_cutoff”은 템플릿/외부정보 사용 시 사실상 안전장치

    temporal_cutoff 같은 컬럼이 있다는 건, 템플릿(구조 DB) 활용을 하더라도 시점 누수(leakage)를 조심하라는 의미로 해석하는 게 안전합니다.

    특히 Part 2는 템플릿 없는 타깃도 포함한다고 알려져 있어(= 템플릿 의존만으로는 한계), 템플릿을 쓰더라도 “없을 때는 다른 모드로” 넘어갈 설계가 중요합니다.

### (C) 회전/이동 불변성 처리

    구조 평가는 보통 회전/이동에 불변이라(최적 정렬 후 비교), 학습은 다음 중 하나로 가는 게 유리합니다.

        SE(3) equivariant 모델(이상적)

        또는 손실함수/증강을 회전 불변으로 구성

            예: 랜덤 회전 증강 + Kabsch 정렬 RMSD / pairwise distance loss

### (D) “best-of-5” 평가를 모델 설계에 반영

    best-of-5면 단일 예측 1개를 극도로 잘 맞추는 것만이 답이 아닙니다.

        5개를 서로 다른 모드로 뽑을수록 “한 개라도 맞을 확률”이 오릅니다.

            예: (1) 템플릿 기반 2개 + (2) 딥러닝 기반 2개 + (3) 구조 다양성용 1개

        단, 5개 모두 질이 너무 낮으면 평균 best-of-5가 크게 안 오르니 **“강한 후보 1~2개 
            + 다양성 3~4개”**가 현실적으로 많이 씁니다.

## 3) 모델 선택: 실전적으로 3가지 트랙(권장 순서)
### 트랙 1) 템플릿 기반(TBM) / 검색-정렬-좌표이식

    장점: 구현 난이도 대비 점수가 빨리 나오는 경우가 많음(특히 유사 서열 존재 시)

    단점: 템플릿이 없는 타깃에 취약(Part 2에서 더 중요)

    best-of-5에 잘 맞음: 서로 다른 템플릿 5개를 그대로 제출 가능

### 트랙 2) 딥러닝 회귀(서열→좌표) + 기하학 손실

    장점: 템플릿 없는 경우에도 대응 가능

    단점: 학습/튜닝 비용 큼, 데이터/시간 제한(노트북 런타임) 고려 필요

    추천 구성(현실적):

        입력: one-hot(ACGU) + (가능하면) MSA 요약 특징(PSSM 등)

        모델: Transformer(1D) + pairwise attention(2D) 또는 GNN

        손실: Kabsch RMSD + 거리행렬 loss + (선택) smoothness

### 트랙 3) 하이브리드(템플릿 + 딥러닝) / 앙상블

    실제로 성능이 잘 나오는 방향: 템플릿 파이프라인, MSA, 프리트레인 foundation model을 섞는 접근이 강력하다는 흐름이 있습니다
        (예: RNAPro는 template modeling + MSA + pretrained RNA model 결합을 강조).

    베이스라인 단계에서도 best-of-5 때문에 하이브리드가 특히 잘 맞습니다.

## 4) 예제 코드 1 — 데이터 로딩 & 전처리(타깃 단위로 묶기)

    아래 코드는 (1) ID 파싱, (2) resid 정렬, (3) target_id → (sequence, coords) 딕셔너리 구성까지 한 번에 합니다.

In [ ]:
import os
import numpy as np
import pandas as pd

DATA_DIR = "/kaggle/input/stanford-rna-3d-folding-2/"

train_seqs = pd.read_csv(os.path.join(DATA_DIR, "train_sequences.csv"))
train_labels = pd.read_csv(os.path.join(DATA_DIR, "train_labels.csv"))

valid_seqs = pd.read_csv(os.path.join(DATA_DIR, "validation_sequences.csv"))
valid_labels = pd.read_csv(os.path.join(DATA_DIR, "validation_labels.csv"))

test_seqs  = pd.read_csv(os.path.join(DATA_DIR, "test_sequences.csv"))
sample_sub = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))

def split_id(id_str: str):
    # ID format: "{target_id}_{residue_number}"
    tid, resid = id_str.rsplit("_", 1)
    return tid, int(resid)

def detect_xyz_cols(df: pd.DataFrame):
    # train/valid labels: usually x,y,z (single structure)
    if set(["x", "y", "z"]).issubset(df.columns):
        return ["x", "y", "z"]
    # sometimes labels already have suffix (rare), fallback:
    if set(["x_1", "y_1", "z_1"]).issubset(df.columns):
        return ["x_1", "y_1", "z_1"]
    raise ValueError(f"Cannot find xyz columns. columns={df.columns.tolist()[:30]}...")

XYZ_COLS = detect_xyz_cols(train_labels)

# target_id -> sequence (and metadata)
seq_map = train_seqs.set_index("target_id")["sequence"].to_dict()
valid_seq_map = valid_seqs.set_index("target_id")["sequence"].to_dict()

def build_coords_dict(labels_df: pd.DataFrame, seq_map: dict):
    coords_dict = {}
    # parse target_id/resid
    tmp = labels_df.copy()
    tmp[["target_id", "resid"]] = tmp["ID"].apply(lambda s: pd.Series(split_id(s)))

    for tid, g in tmp.groupby("target_id", sort=False):
        if tid not in seq_map:
            continue
        g = g.sort_values("resid")
        seq = seq_map[tid]
        L = len(seq)

        # Basic sanity checks
        # resid should be 1..L (or at least within range)
        if g["resid"].min() != 1 or g["resid"].max() != L or len(g) != L:
            # 대회 데이터에 따라 드물게 결측/불일치가 있을 수 있어,
            # 여기서는 raise 대신 continue/보정 로직을 넣을 수도 있음.
            raise ValueError(f"[{tid}] resid range/len mismatch: "
                             f"min={g['resid'].min()}, max={g['resid'].max()}, rows={len(g)}, L={L}")

        coords = g[XYZ_COLS].to_numpy(np.float32)  # (L,3)

        # resname 검증(컬럼이 있을 때만)
        if "resname" in g.columns:
            ok = (g["resname"].astype(str).str.upper().values == np.array(list(seq)))
            if not ok.all():
                raise ValueError(f"[{tid}] resname != sequence at some positions")

        coords_dict[tid] = coords
    return coords_dict

train_coords = build_coords_dict(train_labels, seq_map)
valid_coords = build_coords_dict(valid_labels, valid_seq_map)

print("train targets:", len(train_coords), "valid targets:", len(valid_coords))


## 5) 예제 코드 2 — “일단 제출이 되는” 초간단 베이스라인 (A-form 유사 헬릭스 5개)

    이 코드는 학습 없이도 submission.csv를 만들고, 포맷/ID 매칭 문제를 빠르게 잡는 용도입니다.
    (점수는 높지 않겠지만, 파이프라인 검증에 매우 유용합니다.)

In [ ]:
import numpy as np
import pandas as pd
import os

DATA_DIR = "/kaggle/input/stanford-rna-3d-folding-2/"
test_seqs  = pd.read_csv(os.path.join(DATA_DIR, "test_sequences.csv"))
sub = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))

def helix_coords(L, radius=10.0, rise=2.8, twist=2*np.pi/11.0, phase=0.0, bend=0.0):
    i = np.arange(L, dtype=np.float32)
    theta = phase + twist * i
    x = radius * np.cos(theta)
    y = radius * np.sin(theta)
    z = rise * i
    if bend != 0.0 and L > 1:
        x = x + bend * np.sin(2*np.pi * i / (L-1))
    return np.stack([x, y, z], axis=1)  # (L,3)

# target_id -> 5 predictions (list of (L,3))
preds = {}
for _, row in test_seqs.iterrows():
    tid = row["target_id"]
    seq = row["sequence"]
    L = len(seq)

    # 5개의 구조를 "완전히 동일"하게 내면 best-of-5 이점이 없어서,
    # 파라미터를 조금씩 바꿔 다양성만 최소한 확보
    variants = [
        helix_coords(L, radius=10.0, rise=2.8, twist=2*np.pi/11.0, phase=0.0, bend=0.0),
        helix_coords(L, radius=9.5, rise=2.7, twist=2*np.pi/11.5, phase=0.5, bend=1.0),
        helix_coords(L, radius=10.5, rise=2.9, twist=2*np.pi/10.5, phase=1.0, bend=0.5),
        helix_coords(L, radius=8.5, rise=3.1, twist=2*np.pi/12.0, phase=1.5, bend=1.5),
        helix_coords(L, radius=11.0, rise=2.6, twist=2*np.pi/10.8, phase=2.0, bend=0.8),
    ]
    preds[tid] = variants

# submission 채우기
# sample_submission에는 보통 ID/resname/resid + x_1..z_5 컬럼이 이미 존재
for k in range(1, 6):
    for c in ["x", "y", "z"]:
        col = f"{c}_{k}"
        if col not in sub.columns:
            sub[col] = 0.0

# ID -> (target_id, resid)
tid_res = sub["ID"].apply(lambda s: pd.Series(s.rsplit("_", 1)))
sub["_tid"] = tid_res[0].values
sub["_resid"] = tid_res[1].astype(int).values

# 좌표 대입
for idx, r in sub.iterrows():
    tid = r["_tid"]
    j = r["_resid"] - 1  # 1-indexed -> 0-indexed
    for k in range(5):
        xyz = preds[tid][k][j]
        sub.at[idx, f"x_{k+1}"] = float(xyz[0])
        sub.at[idx, f"y_{k+1}"] = float(xyz[1])
        sub.at[idx, f"z_{k+1}"] = float(xyz[2])

sub.drop(columns=["_tid", "_resid"]).to_csv("submission.csv", index=False)
print("saved submission.csv", sub.shape)



## 6) 예제 코드 3 — (추천) “train set을 템플릿 풀로” top-5 템플릿 매칭해서 best-of-5 노리기

    Part 2는 best-of-5라서, 가장 구현 대비 효율이 좋은 시작점이:

        test 서열마다 train 서열 중 유사한 것 5개를 찾고

        각 템플릿의 C1′ 좌표를 정렬된 resid 기준으로 이식해 5개 구조로 제출

    아래는 “빠르게 돌아가는” 간이 템플릿 매칭 버전(정교한 정렬 대신 k-mer 유사도)입니다.

In [ ]:
import numpy as np
import pandas as pd
import os

DATA_DIR = "/kaggle/input/stanford-rna-3d-folding-2/"

train_seqs = pd.read_csv(os.path.join(DATA_DIR, "train_sequences.csv"))[["target_id","sequence"]]
train_labels = pd.read_csv(os.path.join(DATA_DIR, "train_labels.csv"))
test_seqs  = pd.read_csv(os.path.join(DATA_DIR, "test_sequences.csv"))
sub = pd.read_csv(os.path.join(DATA_DIR, "sample_submission.csv"))

def split_id(id_str: str):
    tid, resid = id_str.rsplit("_", 1)
    return tid, int(resid)

def detect_xyz_cols(df):
    return ["x","y","z"] if set(["x","y","z"]).issubset(df.columns) else ["x_1","y_1","z_1"]

XYZ = detect_xyz_cols(train_labels)

# train target -> coords (L,3)
tmp = train_labels.copy()
tmp[["target_id","resid"]] = tmp["ID"].apply(lambda s: pd.Series(split_id(s)))
train_coords = {}
for tid, g in tmp.groupby("target_id", sort=False):
    g = g.sort_values("resid")
    train_coords[tid] = g[XYZ].to_numpy(np.float32)

train_seq_map = train_seqs.set_index("target_id")["sequence"].to_dict()

def kmer_set(seq, k=3):
    if len(seq) < k:
        return {seq}
    return {seq[i:i+k] for i in range(len(seq)-k+1)}

train_kmers = {tid: kmer_set(seq, 3) for tid, seq in train_seq_map.items()}

def jaccard(a, b):
    inter = len(a & b)
    union = len(a | b)
    return inter / union if union else 0.0

def helix_fallback(L):
    # fallback 구조 (템플릿 매칭 실패/길이 불일치 대응)
    i = np.arange(L, dtype=np.float32)
    return np.stack([10*np.cos(2*np.pi*i/11), 10*np.sin(2*np.pi*i/11), 2.8*i], axis=1)

# 제출용 컬럼 확보
for k in range(1, 6):
    for c in ["x","y","z"]:
        col = f"{c}_{k}"
        if col not in sub.columns:
            sub[col] = 0.0

# ID 파싱
tid_res = sub["ID"].apply(lambda s: pd.Series(s.rsplit("_", 1)))
sub["_tid"] = tid_res[0].values
sub["_resid"] = tid_res[1].astype(int).values

# test마다 top-5 템플릿 찾기
for _, row in test_seqs.iterrows():
    tid = row["target_id"]
    qseq = row["sequence"]
    qkm = kmer_set(qseq, 3)

    # 유사도 계산 (훈련 타깃 전체를 다 돌리면 느릴 수 있음 → 실제론 후보를 줄이는 인덱싱을 추천)
    scores = []
    for ttid, tkm in train_kmers.items():
        scores.append((jaccard(qkm, tkm), ttid))
    scores.sort(reverse=True)
    top = [ttid for _, ttid in scores[:5]]

    # 5개 구조 만들기: 길이가 다르면 fallback
    pred5 = []
    for ttid in top:
        tcoords = train_coords.get(ttid, None)
        if tcoords is None or tcoords.shape[0] != len(qseq):
            pred5.append(helix_fallback(len(qseq)))
        else:
            pred5.append(tcoords.copy())
    # 혹시 부족하면 채우기
    while len(pred5) < 5:
        pred5.append(helix_fallback(len(qseq)))

    # sub에 채우기(해당 tid 행만)
    mask = (sub["_tid"] == tid)
    idxs = sub.index[mask]
    for idx in idxs:
        j = int(sub.at[idx, "_resid"]) - 1
        for k in range(5):
            xyz = pred5[k][j]
            sub.at[idx, f"x_{k+1}"] = float(xyz[0])
            sub.at[idx, f"y_{k+1}"] = float(xyz[1])
            sub.at[idx, f"z_{k+1}"] = float(xyz[2])

sub.drop(columns=["_tid","_resid"]).to_csv("submission.csv", index=False)
print("saved submission.csv", sub.shape)


이 템플릿 방식은 “정렬/길이 맞추기”를 정교하게 할수록 성능이 오릅니다.
다음 업그레이드 순서 추천:

    1. k-mer → 서열 정렬(갭 포함) 기반 좌표 매핑

    2. 갭 구간은 선형 보간 + 스무딩

    3. top-5 템플릿의 좌표를 그대로 제출하는 대신, 일부는 노이즈/국소 변형으로 다양성 추가(best-of-5 최적화)

## 7) (선택) 딥러닝으로 가려면: “최소한 이 구조”를 추천
학습 목표를 TM-score에 더 가깝게

    TM-score 자체를 미분가능하게 넣긴 어렵기 때문에, 보통은

    Kabsch 정렬 RMSD(전역 정렬)

    pairwise distance matrix loss(회전 불변, 전역 형태 학습에 도움)
    
    연속 잔기 간 smoothness(구조가 찢어지는 것 방지) 조합이 무난합니다.

best-of-5 생성 전략

    같은 모델이라도 5개를 만드는 방법:

    1. 앙상블(서로 다른 seed/fold 5개)

    2. MC Dropout 샘플링(inference 때 dropout on)

    3. 노이즈 조건부/디퓨전류 생성(난이도↑)

### MSA 활용(있다면)

    데이터에 MSA가 포함된 경우(파일이 많이 존재)라면, 최소한 PSSM/빈도 특징만 넣어도 종종 도움이 됩니다.

## 8) 마지막으로, 지금 당장 추천하는 진행 순서(실전 플로우)

    1. **예제 코드 2(A-form helix)**로 submission.csv 생성 → 제출 포맷/ID 매칭 문제 0으로 만들기

    2. **예제 코드 3(템플릿 top-5)**로 점수 빠르게 올리기(best-of-5를 제대로 활용)

    3. 그 다음에 딥러닝을 붙이되,

        템플릿이 있는 경우: 템플릿을 1~2개는 항상 후보로 남기고

        템플릿이 없는 경우: 딥러닝 예측이 best-of-5에서 살아남도록 “다양성 생성”을 설계
        (Part 2는 템플릿 없는 타깃이 포함된다고 알려져 이 분기가 중요합니다.)